In [1]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
from functions import utils
import json
import warnings
warnings.filterwarnings("ignore")

c:\Users\isakn\miniconda3\envs\SML_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = utils.secrets_api()

city_secrets = json.loads(secrets.get_secret("city_secrets").value)
CITY = city_secrets["city"]
latitude = city_secrets["latitude"]
longitude = city_secrets["longitude"]

time_secrets = json.loads(secrets.get_secret("time_secrets").value)
earliest_bike_datetime = time_secrets["earliest_bike_datetime"]
last_bike_datetime = time_secrets["last_bike_datetime"]

today = datetime.now()

print(type(fs))

2025-01-07 09:59:18,600 INFO: Initializing external client
2025-01-07 09:59:18,601 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 09:59:20,033 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
2025-01-07 09:59:21,381 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 09:59:21,386 INFO: Initializing external client
2025-01-07 09:59:21,386 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 09:59:22,657 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
2025-01-07 09:59:23,882 INFO: Closing external client and cleaning up certificates.
2025-01-07 09:59:23,886 INFO: Initializing external client
2025-01-07 09:59:23,887 INFO: Base URL: https://c.app.hopsworks.ai:443
<class 'hsfs.feature_store.FeatureStore'>


## Creating the feature view

In [ ]:
bike_fg = fs.get_feature_group("bike_data")

TypeError: FeatureStore.get_feature_group() missing 1 required positional argument: 'name'

In [ ]:
selected_features = bike_fg.select(["station", "num_bikes_available"]).join(weather_fg.select_all(), on=['datetime'])
selected_features.show(10)

"""
bike_df = bike_fg.read()
weather_df = weather_fg.read()

# Merge data
bike_df['datetime'] = pd.to_datetime(bike_df['datetime'])
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

df = pd.merge(bike_df, weather_df, on='datetime', how='left')

df
"""

In [6]:
feature_view = fs.get_or_create_feature_view(
    name='bike_weather_fw',
    description="weather features with bike availability as the target",
    version=1,
    labels=['num_bikes_available'],
    query=selected_features,
)


In [7]:
start_date_test_data = "2024-10-15"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

In [ ]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.2, 
)

In [ ]:
X_train